In [ ]:
#Bloque0 Instalación de la libreria pydeck
!pip install pydeck

In [3]:
#RECOMENDACIONES:
#Una vez bajado el proyecto, dirijirse a la documentación de pydeck e instalar todas las dependecias necesarias.
#Recomendamos abrir el proyecto en jupiter o google colab para evitar dificultades.

# Bloque1 (Importación de librerias y de datos)
import pandas as pd
import pydeck as pdk
import numpy as np
import networkx as nx
archivo=pd.read_csv("https://raw.githubusercontent.com/mauriciotoro/ST0245-Eafit/master/proyecto/Datasets/calles_de_medellin_con_acoso.csv", sep=";")
poligono=pd.read_csv("https://raw.githubusercontent.com/mauriciotoro/ST0245-Eafit/master/proyecto/Datasets/poligono_de_medellin.csv", sep=";")



In [13]:
#Bloque 2 (llenar vacios con ceros)
archivos=archivo.fillna(0)

In [ ]:
#Bloque 3 (Sacar promedio de acoso y distancia, a traves de  la multiplicación)
archivos['promedio']=archivos.length*archivos.harassmentRisk
archivos

In [ ]:
#Bloque 4 (Medir zonas con más alto riesgo)
acoso=archivo['harassmentRisk']>0.97
archivo2=archivo[acoso]

#Eliminamos columnas sobrantes
del(archivo2['name'])
del(archivo2['length'])
del(archivo2['oneway'])
del(archivo2['harassmentRisk'])
del(archivo2['geometry'])

#divimos las longitudes y latitudes de origen y destino 
archivo2['longitud_d']=archivo2['destination'].map(lambda x:x.split(',')[0])
archivo2['latitud_d']=archivo2['destination'].map(lambda x:x.split(',')[1])
archivo2['longitud_o']=archivo2['origin'].map(lambda x:x.split(',')[0])
archivo2['latitud_o']=archivo2['origin'].map(lambda x:x.split(',')[1])

#borramos las columnas de origin y destination
del(archivo2['origin'])
del(archivo2['destination'])
#eliminamos los parentesis y los pasamos a float
archivo2=archivo2.replace('\(|\)','',regex=True).astype(float)

#Nueva lista que indica el color de las calles de alto riesgo de acoso
color = []
for _ in range(7576):
   color.append("#ed1c24")

#Nueva lista con el nommbre que aparecera con pongamos el mouse sobre la ruta
name = []
for _ in range(7576):
   name.append("Alto riesgo de acoso")

#Lista final con el mayor número de acoso en las calles
acoso_o = list(zip(archivo2.longitud_o,archivo2.latitud_o))
acoso_d = list(zip(archivo2.longitud_d,archivo2.latitud_d))
acoso_M=list(zip(acoso_o,acoso_d))
mayor_acoso=list(zip(name,color,acoso_M))
mayor_acoso

In [ ]:
#Bloque 5 (organización de información y coordenadas del usuario)
distancia = nx.from_pandas_edgelist(archivos, source='origin', target='destination', edge_attr='length')
acoso = nx.from_pandas_edgelist(archivos, source='origin', target='destination', edge_attr='harassmentRisk')
promedio = nx.from_pandas_edgelist(archivos, source='origin', target='destination', edge_attr='promedio')

distancia

origin_long=float(input("Ingrese longitud del origen: "))
origin_lat=float(input("Ingrese latitud del origen: "))

#Coordenadas de origen
origen=(origin_long, origin_lat)

destination_long=float(input("Ingrese longitud del destino: "))
destination_lat=float(input("Ingrese latitud del destino: "))

#Coordenadas de destino
destino=(destination_long,destination_lat)



In [ ]:
#Bloque6 (Imprimir vertices o nodos de la grafica)
distancia.nodes()

In [ ]:
#Bloque7 (Imprimir las aristas de las distancias)
distancia.edges()

In [ ]:
#Bloque8 (Cantidad de vertices o nodos)
distancia.order()

In [ ]:
#Bloque9 (Implementacion de algoritmo dijkstra para ruta más corta en una lista de strings)
djk_distancia=nx.dijkstra_path(distancia, source=str(origen), target=str(destino), weight='length')
long_distancia=nx.dijkstra_path_length(distancia, source=str(origen), target=str(destino), weight=True)
djk_distancia


In [ ]:
#Bloque11 (Implementacion de algoritmo dijkstra para ruta con menos acoso en una lista de strings)
djk_acoso=nx.dijkstra_path(acoso, source=str(origen), target=str(destino), weight='harassmentRisk')
lon_acoso=nx.dijkstra_path_length(acoso, source=str(origen), target=str(destino), weight='harassmentRisk')
djk_acoso

In [ ]:
#Bloque12 (Implementacion de algoritmo dijkstra para el promedio de rutas en una lista de strings)
djk_promedio=nx.dijkstra_path(promedio, source=str(origen), target=str(destino), weight='promedio')
lon_promedio=nx.dijkstra_path_length(promedio, source=str(origen), target=str(destino), weight='promedio')
djk_promedio

In [ ]:
#Bloque13
#Pasamos a dataframe el resultado
dist = pd.DataFrame(djk_distancia,columns=['calles'])

# dividimos en dos columnas la longitud y la latitud
dist['longitud']=dist['calles'].map(lambda x:x.split(',')[0])
dist['latitud']=dist['calles'].map(lambda x:x.split(',')[1])

#borramos la columna de calles
del(dist['calles'])

#eliminamos los parentesis y los pasamos a float
dist=dist.replace('\(|\)','',regex=True).astype(float)

#hacemos una nueva lista con el sistemas de coordenadas tipo turple
points = list(zip(dist.longitud,dist.latitud))
points

In [ ]:
#Bloque14 (Nueva lista con información de ruta mas corta)
calles=[points]
name=["Distancia más corta, con una longitud de: " + str(long_distancia) + " calles"]
color=["#ff0080"]
camino=list(zip(name,color,calles))

camino

In [ ]:
#Bloque15
#Pasamos a dataframe el resultado
acs = pd.DataFrame(djk_acoso,columns=['ruta'])

# dividimos en dos columnas la longitud y la latitud
acs['longitud']=acs['ruta'].map(lambda x:x.split(',')[0])
acs['latitud']=acs['ruta'].map(lambda x:x.split(',')[1])

#borramos la columna de calles
del(acs['ruta'])

#eliminamos los parentesis y los pasamos a float
acs=acs.replace('\(|\)','',regex=True).astype(float)

#hacemos una nueva lista con el sistemas de coordenadas tipo turple
puntos = list(zip(acs.longitud,acs.latitud))
puntos

In [ ]:
#Bloque16 (Nueva lista con información de ruta mas corta)
calles_acoso=[puntos]
name=["Distancia con menos acoso, con una longitud de: " + str(lon_acoso) + " de acoso"]
color=["#FFEB00"]
camino_acoso=list(zip(name,color,calles_acoso))

camino_acoso

In [ ]:
#Bloque17
#Pasamos a dataframe el resultado
prom = pd.DataFrame(djk_promedio,columns=['suma'])

# dividimos en dos columnas la longitud y la latitud
prom['longitud']=prom['suma'].map(lambda x:x.split(',')[0])
prom['latitud']=prom['suma'].map(lambda x:x.split(',')[1])

#borramos la columna de calles
del(prom['suma'])

#eliminamos los parentesis y los pasamos a float
prom=prom.replace('\(|\)','',regex=True).astype(float)

#hacemos una nueva lista con el sistemas de coordenadas tipo turple
pus = list(zip(prom.longitud,prom.latitud))
pus

In [ ]:
#Bloque18 (Nueva lista con información de ruta mas corta)
calles_prom=[pus]
name=["Distancia de promedio de acoso y distancia, con una longitud de: " + str(lon_promedio) + " promedio"]
color=["#03972D"]
suma=list(zip(name,color,calles_prom))

suma

In [ ]:
#Bloque 19 (Nuevo dataframe de las rutas de mayor acoso)
ruta_acoso=pd.DataFrame(mayor_acoso, columns=['name','color','path'])
ruta_acoso

In [ ]:
#Bloque20 Nuevo dataframe para colorear la ruta más corta
distancia_corta = pd.DataFrame(camino, columns=['name','color','path'])
distancia_corta

In [ ]:
#Bloque21 Nuevo dataframe para colorear la ruta con menos acoso
distancia_acoso = pd.DataFrame(camino_acoso, columns=['name','color','path'])
distancia_acoso

In [ ]:
#Bloque22 Nuevo dataframe para colorear la ruta con promedio
dista_prom = pd.DataFrame(suma, columns=['name','color','path'])
dista_prom

In [ ]:
#Bloque23 Unión de los dataframe, para exportar datos en el mapa
rutas = [distancia_corta, distancia_acoso, dista_prom, ruta_acoso]
final=pd.concat(rutas)
final

In [ ]:
#Bloque24 (Exportar mapa)

#Función que lee el color de la ruta y la pinta
def ruta_color(d):
    d = d.lstrip("#")
    return tuple(int(d[i : i + 2], 16) for i in (0, 2, 4))


final["color"] = final["color"].apply(ruta_color)

#Coordenadas en donde aparece el mapa en el lugar de origen
coordenadas_map = pdk.ViewState(latitude=origin_lat, longitude=origin_long, zoom=18, bearing=0, pitch=45)

#Caracteristicas del mapa
layer = pdk.Layer(
    type="PathLayer",
    data=final,
    pickable=True,
    get_color="color",
    width_scale=2,
    width_min_pixels=2,
    get_path="path",
    get_width=2,
)  

#extraemos propiedades del mapa
mapa = pdk.Deck(layers=[layer], initial_view_state = coordenadas_map, tooltip={"text": "{name}"})

#pasamos el mapa a html
mapa.to_html("map.html")